In [16]:
import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
def read_movie_as_dataframe():
    ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
    movies = pd.read_csv('../data/ml-latest-small/movies.csv')
    return ratings,movies

In [40]:
def read_movie():
    data=pd.read_csv("../data/ml-20m/ratings.csv")
    genre_data=pd.read_csv("../data/ml-20m/movies.csv")
    movie_data_ratings_data=data.merge(genre_data,on = 'movieId',how = 'inner')

    movie_data_ratings_data['timestamp'] = pd.to_datetime(movie_data_ratings_data['timestamp'],unit='s')
    movie_data_ratings_data['timestamp'] = movie_data_ratings_data['timestamp'].dt.year

    subset_data=movie_data_ratings_data[movie_data_ratings_data['timestamp']==2015]
    year=movie_data_ratings_data['timestamp'][0]
    b=pd.Series(subset_data.userId).value_counts().rename_axis('user').reset_index()
    c=pd.Series(subset_data.movieId).value_counts().rename_axis('movie').reset_index()
    subset_data_2=subset_data[subset_data.userId.isin(b[b.userId>30].user) & subset_data.movieId.isin(c[(c.movieId>100)].movie)].reset_index(drop=True)
    
    ratings = subset_data_2[subset_data_2['timestamp']==2015][['userId','movieId','rating']]
    movies = subset_data_2[subset_data_2['timestamp']==2015][['movieId','title','genres']]
    return ratings, movies

In [50]:
def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [51]:
def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [52]:
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [53]:
def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    
    train_precision = precision_at_k(model, x, k=2).mean()
    test_precision = precision_at_k(model, x, k=2).mean()
    train_auc = auc_score(model, x).mean()
    test_auc = auc_score(model, x).mean()
    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
    
    return model

In [54]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list

In [55]:
def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 



In [56]:
def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

In [57]:
def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest :{0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [58]:
if __name__=='__main__':
    ratings, movie = read_movie_as_dataframe()
#     ratings, movie = read_movie()
    interactions = create_interaction_matrix(df = ratings,
                                         user_col = 'userId',
                                         item_col = 'movieId',
                                         rating_col = 'rating')

    # Create User Dict
    user_dict = create_user_dict(interactions=interactions)
    # Create Item dict
    movies_dict = create_item_dict(df = movie,id_col = 'movieId',name_col = 'title')
    mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

Precision: train 0.71, test 0.71.
AUC: train 0.99, test 0.99.


In [67]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [68]:
 movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [69]:
interactions.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 5378,
                                    item_dict = movies_dict,
                                    n_items = 10)
print("Recommendation list is")
print(rec_list)

Item of interest :Star Wars: Episode II - Attack of the Clones (2002)
Item similar to the above item:
1- Spider-Man (2002)
2- Minority Report (2002)
3- Signs (2002)
4- X2: X-Men United (2003)
5- X-Men (2000)
6- Charlie's Angels (2000)
7- Spider-Man 2 (2004)
8- Star Wars: Episode III - Revenge of the Sith (2005)
9- Last Samurai, The (2003)
10- Star Wars: Episode I - The Phantom Menace (1999)
Recommendation list is
[5349, 5445, 5502, 6333, 3793, 3977, 8636, 33493, 7143, 2628]


In [71]:
#     train_precision = precision_at_k(mf_model, train, k=10).mean()
#     test_precision = precision_at_k(mf_model, test, k=10).mean()
#     train_auc = auc_score(model, train).mean()
#     test_auc = auc_score(model, test).mean()
#     print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
#     print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

rec_list = sample_recommendation_user(model = mf_model, 
                                  interactions = interactions, 
                                  user_id = 11, 
                                  user_dict = user_dict,
                                  item_dict = movies_dict, 
                                  threshold = 4,
                                  nrec_items = 10,
                                  show = True)

Known Likes:
1- Saving Private Ryan (1998)
2- As Good as It Gets (1997)
3- Titanic (1997)
4- Amistad (1997)
5- Contact (1997)
6- Last of the Mohicans, The (1992)
7- Top Gun (1986)
8- Silence of the Lambs, The (1991)
9- Searching for Bobby Fischer (1993)
10- Fugitive, The (1993)
11- Forrest Gump (1994)
12- Clear and Present Danger (1994)
13- Apollo 13 (1995)
14- Braveheart (1995)
15- Heat (1995)

 Recommended Items:
1- Terminator 2: Judgment Day (1991)
2- Independence Day (a.k.a. ID4) (1996)
3- Die Hard: With a Vengeance (1995)
4- Speed (1994)
5- Star Wars: Episode VI - Return of the Jedi (1983)
6- Star Trek: First Contact (1996)
7- Rock, The (1996)
8- Star Wars: Episode IV - A New Hope (1977)
9- Stargate (1994)
10- Batman (1989)


In [ ]:
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)

print()

rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                item_id = 5378,
                                item_dict = movies_dict,
                                n_items = 10)
print("Recommendation list is")
print(rec_list)

In [ ]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 5378,
                                    item_dict = movies_dict,
                                    n_items = 10)